# 3.3 可変データによるモデル化

今、データは第二章以前では触れなかった別の側面があることがわかっている。それは**状態が変化する**、ということです。

状態が変化する複合オブジェクトをモデル化するために、

1. セレクタ
2. コンストラクタ

に加えて、データオブジェクトを変更する

3. ミューテータ(*mutator*)

と呼ばれる演算子を含めたデータ抽象化を設計する。



ミューテータが定義されているデータオブジェクトは、可変データオブジェクト(mutable data object)と呼ばれている。


## 3.3.1 可変リスト構造

ペアに対する基本演算、　*cons, car, cdr* は、リスト構造を構築したり、そのリスト構造から部品（一部）を選択するために使うことができる。しかし、これらはリスト構造を変化することはできません。(append, list のようなリストの演算についても同様)なぜなら、*cons, car, cdr*によって定義できるものだからです。

リスト構造を変化するには、新しい命令が必要となる。ペアに対する基本ミューテータは
1. set-car! ... ２つの引数を取り、１つ目の引数はペアでなければなりません。set-car!はこのペアを変更し、 carポインタをset-car!の第二引数へのポインタに置き換えます。
2. set-cdr! ... (set-car! の　car部分をcdrにする)

です。

```scheme
(define x '((a b) c d))
(define y '(e f))
; とする。
(set-car! x y)
; を実行すると、
x ; => ((e f) c d)

```


あとは、既存のリスト構造の一部でない新しいペアを返す*get-new-pair*
という手続きがあれば、それらによってconsを実装することができる。
```scheme
(define (cons x y)
    (let ((new (get-new-pair)))
        (set-car! new x)
        (set-cdr! new y)
     new))
```

---

#### Ex. 3.12

appned!を定義する。これはコンストラクタでは無くミューテータである。

```scheme
(define (append! x y)
    (set-cdr! (last-pair x) y)
    x)
    
; 参考までに
(define (append x y)
    (if (null? x)
        y
        (cond (car x) (append (cdr x) y))))
```

last-pair手続きを定義する。これは引数の最後のペアを返す。

```scheme
(define (last-pair x)
    (if (null? (cdr x))
    x
    (last-pair (cdr x))))
```

問題

```scheme
> (define x (list 'a 'b))
> (define y (list 'c 'd))
> (define z (append x y))

;----

> z
;=> (a b c d)

> (cdr x)
;=> (b)

> (define w (append! x y))
> w
;=> (a b c d)

> (cdr x)
;=> (b c d)

```

---

#### Ex. 3.13

make-cycle手続きについて考える。

```scheme
(define (make-cycle x)
    (set-cdr! (last-pair x) x)
    x)
```

次の式によって作られる構造zを表す箱とポインタの図を描け。


```scheme
(define z (make-cycle (list 'a 'b 'c)))
```

---

```
 -----------------------------
 |                           |
 --> |●|●|--->|●|●|--->|●|●|--
      |        |        |
      v        v        v
      a        b        c


null -> 先頭へのポインタ
```

---

```scheme

(last-pair z)

```
は無限にループ

---

#### Ex. 3.14

loop は一時的な変数tempを使って、xのcdrの古い値を保持する。これは、次の行のset-cdr!がcdrを破壊するためである。一般的にmysteryが何をするのか説明せよ。


v は (define v (list 'a 'b 'c 'd))によって定義されたものだとする。　ｖが束縛されるリストを表す箱とポインタの図を描け。次に　(define w (mystery v)) を評価したとする。この式を評価した後のvとwの構造を表す箱とポインタの図を描け。vとwの値として何が表示されるだろうか。


```scheme
(define (mystery x)
    (define (loop x y)
        (if (null? x)
            y
            (let ((temp (cdr x)))
                (set-cdr! x y)
                (loop temp x))))
     (loop x '()))
```

##### V

```
                           
v --> |●|●|--->|●|●|--->|●|●|--->|●|/|
      |        |        |        |
      v        v        v        v
      a        b        c        d

```

##### W 

```
w --> |●|●|--->|●|●|--->|●|●|--->|●|/|
      |        |        |        |
      v        v        v        v
      d        c        b        a


```


---




### 共有とアイデンティティ

代入を導入することによって出て来る"同一性"と"変更"という理論的な問題について触れた。
これらの問題は、個々のペアが別々のデータオブジェクトの間で共有されると実際に怒るようになる。

```scheme

(define x (list 'a 'b))
(define z1 (cons x x)) ; car と cdr によって x が共有できるようになっている。
(define z2 (cons (list 'a 'b) (list 'a 'b)))

(define (set-to-wow! x) (set-car! (car x) 'wow) x)


> z1
((a b) a b)

>(set-to-wow! z1)
((wow b) wow b)

> z2
((a b) a b)

> (set-to-wow! z2)
((wow b) a b)


```

(eq? x y)はxとyが同じオブジェクトかどうか（つまり、xとyがポインタとして同じか）をテストする。

---

#### Ex. 3.15

---

```
z1 -->|●|●|
       | |
       v v
 x -->|●|●|--->|●|/|
       |        |   
       v        v
       a        b
       
> (set-to-wow! z1)


z1 -->|●|●|
       | |
       v v
 x -->|●|●|--->|●|/|
       |        |   
       v        v
      wow       b
       

```

---

```

       
z2 -->|●|●|--->|●|●|--->|●|\|
       |        |        |   
       |        v        v  
       |                    
       |        a        b            
       |                              
       |        ^        ^             
       |        |        |              
       |------>|●|●|--->|●|\|
       



> (set-to-wow! z2)

       
z2 -->|●|●|--->|●|●|--->|●|\|
       |        |        |   
       |        v        v  
       |        a            
       |                 b            
       |       wow                      
       |        ^        ^             
       |        |        |              
       |------>|●|●|--->|●|\|



```

---

#### Ex. 3.16

```scheme
(define (count-pairs x)
    (if (not (pair? x))
    0
    (+ (count-pairs (car x))
    (count-pairs (cdr x))
    1)))
```

- 3: 通常のリスト
- 4:
    ```scheme
    (define w1 (cons 'a (cons 'b nil)))
    (define z4 (cons w1 (cdr w1)))
    ```
- 7:
    ```scheme
    (define a (cons 'a nil))
    (define b (cons a a))
    (define z7 (cons b b))
    ```
- loop: 上でやったmake-cycleを使うやつ







---

#### Ex. 3.17

*set!* を使う。

```scheme
(define (m-count-pairs p)
  (let ((paths '()))
    (define (count p)
      (cond ((null? p) 0)
            ((not (pair? p)) 0)
            ((in p paths) (+ (count (car p)) (count (cdr p))))
            (else (set! paths (cons p paths))
                  (write paths)
                  (+ 1 (count (car p)) (count (cdr p))))))
    (count p)))
```


---

#### Ex. 3.18, Ex.3.19

```
example
1 -> 2 -> 3 -> 4
          ∧    |  
          |    ∨
          6 <- 5

```

```scheme

; <3.18>

(define (loop? l)
  (define (check l2)
    (cond ((null? l2) #f)
          ((eq? l l2) #t)
          (else (check (cdr l2)))))
  (check (cdr l)))

; 途中から循環してる場合は、自分の位置をどこかに保存しておいてから、その位置で上のコードを実行する。
; しかも、上の例だと3~6で位置を保存・上のコードを実行しないとだめ。
; 1,2で上のコードを実行すると、無限ループを検出せずに無限ループに陥る。

; ゴニョゴニョ（省略）



; ただ、上の方法だと位置の保存にメモリを必要とする。
;=> 位置を保存しない方法は無いか?

; 3.19
(define (loop?? l)
  (define (check l2 l3)
    (cond ((null? l2) #f)
          ((eq? l2 l3) #t)
          (else (check (cdr l2) (cddr l3)))))
  (cond ((not (pair? l)) #f)
        ((null? (cddr l)) #f)
        (else (check (cdr l) (cddr l)))))
```  



---

### 変更は代入にすぎない

複合データを導入した時、ペアが純粋に手続きだけによって表現できるということを見た。

```scheme
(define (cons x y)
    (define (dispatch m)
        (cond ((eq? m 'car) x)
              ((eq? m 'cdr) y)
              (else (error "Undefined operation: CONS" m))))
    dispatch)
(define (car z) (z 'car))
(define (cdr z) (z 'cdr))
```


可変データについても同じことが言える。可変データオブジェクトは、代入と局所状態を使って手続きとして実装できます。


```scheme
(define (cons x y)
    (define (set-x! v) (set! x v))
    (define (set-y! v) (set! y v))
    (define (dispatch m)
        (cond ((eq? m 'car) x)
              ((eq? m 'cdr) y)
              ((eq? m 'set-car!) set-x!)
              ((eq? m 'set-cdr!) set-y!)
              (else
                  (error "Undefined operation: CONS" m))))
        dispatch)
(define (car z) (z 'car))
(define (cdr z) (z 'cdr))
(define (set-car! z new-value)
    ((z 'set-car!) new-value) z)
(define (set-cdr! z new-value)
   ((z 'set-cdr) new-value) z)
   
```


---

可変データの振る舞いを説明するためには、理論的には、代入だけあれば十分。


---

#### Ex. 3.20

```scheme

(define x (cons 1 2))
(define z (cons x x))
(set-car! (cdr z) 17)
(car x)
> 17
```

---

## 3.3.2 キューの表現


キュー(queue)とは、項目を一方の端(終端(reaer))に挿入し、もう一方の端（先端(front))から削除する列。

- (make-queue) : コンストラクタ。空のキューを返す。
- (empty-queue? < queue > ) : キューが空かどうかテスト
- (front-queue < queue > ) : キューの先頭にあるオブジェクトを返す。
- (insert-queue! < queue > < item > ) : キューの最後尾にアイテムを挿入し、変更されたキューを値として返す。
- (delete-queue! < queue > ) : キューの先頭のアイテムを削除し、変更されたキューを値として返す。


```scheme
(define (front-ptr queue) (car queue))

(define (rear-ptr queue) (cdr queue))

(define (set-front-ptr! queue item)
  (set-car! queue item))
  
(define (set-rear-ptr! queue item)
  (set-cdr! queue item))
  
(define (empty-queue? queue)
  (null? (front-ptr queue)))
  
(define (make-queue) (cons '() '()))

(define (front-queue queue)
  (if (empty-queue? queue)
      (error "FRONT called with an empty queue" queue)
      (car (front-ptr queue))))

(define (insert-queue! queue item)
  (let ((new-pair (cons item '())))
    (cond ((empty-queue? queue)
           (set-front-ptr! queue new-pair)
           (set-rear-ptr! queue new-pair)
           queue)
          (else
           (set-cdr! (rear-ptr queue) new-pair)
           (set-rear-ptr! queue new-pair)
           queue))))

(define (delete-queue! queue)
  (cond ((empty-queue? queue)
         (error "DELETE! called with an empty queue" queue))
         (else (set-front-ptr! queue (cdr (front-ptr queue)))
               queue)))
```


---


#### Ex. 3.21

雑に...

```scheme
(define (print-queue queue)
  (car queue))
```



#### Ex. 3.22



```scheme
(define (make-queue-dispatch)
  (let ((front-ptr '())
        (rear-ptr '()))
    (define (empty?)
      (null? front-ptr))
    (define (front)
      (if (empty?)
          (error "FRONT called with an empty queue")
          (car front-ptr)))
    (define (print)
      (display front-ptr))
    (define (insert i)
      (let ((new-pair (cons i '())))
        (cond ((empty?) (set! front-ptr new-pair)
                        (set! rear-ptr new-pair))
              (else (set-cdr! rear-ptr new-pair)
                    (set! rear-ptr new-pair)))))
    (define (delete)
      (cond ((empty?) (error "DELETE! called with an empty queue"))
            (else
             (set! front-ptr (cdr front-ptr))
             front-ptr)))
    (define (dispatch m . args)
      (cond ((eq? m 'empty?) (empty?))
            ((eq? m 'front) (front))
            ((eq? m 'insert!) (insert (car args)))
            ((eq? m 'print) (print))
            ((eq? m 'delete!) (delete))))
    dispatch))
            

(define qq (make-queue-dispatch))
(qq 'insert! '1)
(qq 'insert! '2)
(qq 'print)
(newline)
(qq 'delete!)
(qq 'print)


```

#### Ex. 3.23


memo
```haskell
data Queue a = Q [a] [a]

head (Q (x:f) r) = x

tail (Q [x] r) = Q (reverse r) []
tail (Q (x:f) r) = f

snoc (Q [], _) x = Q [x] []
snoc (Q f r) x = Q f (x:r)

-- OR, slightly cleaner way..

checkf (Q [] r) = Q (reverse r) []
checkf q = q

snoc (Q f r) x = checkf (Q f (x:[r]))
tail (Q (x:f) r) = checkf (Q f r)
```



In [8]:
;
(define (make-queue) (cons '() '()))

;
(define (empty-deque? queue)
  (and (null? (front-ptr queue))
       (null? (rear-ptr queue))))

;;
(define (front-ptr queue) (car queue))
;;
(define (rear-ptr queue) (cdr queue))
;;
(define (set-front-deque! queue item)
  (set-car! queue item))
;;
(define (set-rear-deque! queue item)
  (set-cdr! queue item))

;
(define (front-deque dq)
  (if (empty-front-deque? dq)
      (error "FRONT called with an empty queue" dq)
      (car (front-ptr queue))))
;;
(define (empty-front-dequeue?)) ;;;;;;;;

;
(define (rear-deque dq)
  (if (empty-rear-queue? dq)
      (error "FRONT called with an empty queue" dq)
      (car (front-ptr dq))))
;;
(define (empty-raer-dequeue? dq)
  (null? (rear-ptr dq)))



(define (front-insert-queue! queue item)
  (let ((new-pair (cons item '())))
    (cond ((empty-queue? queue)
           (set-front-ptr! queue new-pair)
           (set-rear-ptr! queue new-pair)
           queue)
          (else
           (set-cdr! (rear-ptr queue) new-pair)
           (set-rear-ptr! queue new-pair)
           queue))))
;
(define (rear-insert-deque! queue item)
  (let ((new-pair (cons item '())))
    (set-cdr! new-pair (rear-ptr queue))
    (set-rear-deque! queue new-pair)))
;
(define (rear-delete-deque queue)
  (set-rear-deque! queue (cdr (rear-ptr queue))))

(define (delete-queue! queue)
  (cond ((empty-queue? queue)
         (error "DELETE! called with an empty queue" queue))
         (else (set-front-ptr! queue (cdr (front-ptr queue)))
               queue)))

;(define q (make-queue))
;(insert-queue! q 'a)
;(insert-queue! q 'b)
;(insert-queue! q 'c)
;(delete-queue! q)
;(delete-queue! q)